## Advanced way

- While loop
- Declare variables
- Stored Procedure

In [5]:
CREATE PROCEDURE GetTableRowCounts -- Procedúra készítése, és elnevezése
    @schema_name NVARCHAR(128)  -- A séma neve, amire szeretnél lekérdezni
AS
BEGIN
    DECLARE @table_name NVARCHAR(128);
    DECLARE @row_count INT;
    DECLARE @sql NVARCHAR(MAX);

    -- Kurzor létrehozása, amely az adott séma tábláinak neveit tartalmazza
    DECLARE table_cursor CURSOR FOR
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = @schema_name
      AND table_type = 'BASE TABLE';

    OPEN table_cursor;
    FETCH NEXT FROM table_cursor INTO @table_name;

    -- Ciklus, amely végigmegy az összes táblán
    WHILE @@FETCH_STATUS = 0
    BEGIN
        -- Dinamikus SQL a COUNT(*) végrehajtásához
        SET @sql = 'SELECT @row_count_out = COUNT(*) FROM ' + QUOTENAME(@schema_name) + '.' + QUOTENAME(@table_name);
        
        -- Dinamikus SQL végrehajtása, sorok számának lekérdezése
        EXEC sp_executesql @sql, N'@row_count_out INT OUTPUT', @row_count OUTPUT;

        -- Eredmény kiírása
        PRINT 'Table: ' + @table_name + ', Row Count: ' + CAST(@row_count AS NVARCHAR(20));

        -- Következő tábla
        FETCH NEXT FROM table_cursor INTO @table_name;
    END

    -- Kurzor lezárása és felszabadítása
    CLOSE table_cursor;
    DEALLOCATE table_cursor;
END;


: Msg 2714, Level 16, State 3, Procedure GetTableRowCounts, Line 1
There is already an object named 'GetTableRowCounts' in the database.

Total execution time: 00:00:00.072

## A tárolt eljárás meghívása

In [6]:
-- Példa a 'dbo' séma tábláinak lekérdezésére

USE sql_recap;
EXEC GetTableRowCounts @schema_name = 'dbo';


Table: customers, Row Count: 500

Table: order_details, Row Count: 8981

Table: orders, Row Count: 3000

Table: products, Row Count: 200

Total execution time: 00:00:00.028